In [39]:
import re
from collections import Counter
import numpy as np

In [22]:
FILENAME = 'shakespeare.txt'

In [4]:
with open(FILENAME, 'r') as f:
  content = f.read()
  f.close()

In [5]:
def preprocess_text_data(filename):
  with open(filename, 'r') as f:
    content = f.read()

  words_l = re.findall(r'\b\w+\b', content.lower())

  return words_l

In [24]:
def get_vocab(FILENAME):
  return set(preprocess_text_data(FILENAME))

In [6]:
def get_count_dict(words_l):
  # word_count_dict = {}

  # for word in words_l:
  #   word_count_dict[word] = word_count_dict.get(word, 0) + 1

  # return word_count_dict
  return Counter(words_l)

In [7]:
def get_probs_dict(word_count_dict):
  total_count = 0
  for count in word_count_dict.values():
    total_count += count

  probs_dict = {k : v / total_count for k, v in word_count_dict.items()}

  return probs_dict

In [8]:
def delete_letter(word):
  splits_l = [(word[:i], word[i:]) for i in range(len(word) + 1)]
  delete_l = [l + r[1:] for l, r in splits_l if r]

  return delete_l

In [9]:
def switch_letter(word):
  if len(word) < 2:
    return []

  splits_l = [(word[:i], word[i:]) for i in range(len(word) + 1)]
  switch_l = [l + r[:2][::-1] + r[2:] for l, r in splits_l if len(r) >= 2]

  return switch_l

In [10]:
def replace_letter(word):
  letters = 'abcdefghijklmnopqrstuvwxyz'
  splits_l = [(word[:i], word[i:]) for i in range(len(word) + 1)]
  replace_l = [l + letter + r[1:] for l, r in splits_l for letter in letters if r and r[0] != letter]

  return replace_l

In [11]:
def insert_letter(word):
  letters = 'abcdefghijklmnopqrstuvwxyz'
  splits_l = [(word[:i], word[i:]) for i in range(len(word) + 1)]
  insert_l = [l + letter + r for l, r in splits_l for letter in letters]

  return insert_l

In [12]:
def edit_one_letter(word, allow_switches=False):
  edits = set(delete_letter(word) + replace_letter(word) + insert_letter(word))
  if allow_switches:
    edits.update(switch_letter(word))

  return edits

In [13]:
def edit_two_letters(word, allow_switches=False):
  edits_one_letter = edit_one_letter(word, allow_switches=allow_switches)
  edits_two_letter = set()

  for word_ in edits_one_letter:
    edits_two_letter.update(edit_one_letter(word_, allow_switches=allow_switches))

  return edits_two_letter

In [70]:
def get_corrections(word, probs, vocab, top_n=3):
  suggestions = set()

  if word in vocab:
    suggestions.update([word])
  else:
    one_letter_edits = edit_one_letter(word,
                                       allow_switches=True).intersection(vocab)
    if one_letter_edits:
      suggestions.update(one_letter_edits)
    else:
      two_letter_edits = edit_two_letters(word,
                                          allow_switches=True).intersection(vocab)
      if two_letter_edits:
        suggestions.update(two_letter_edits)
      else:
        suggestions.update(word)

  suggestions_probs = {word: probs.get(word, 0) for word in suggestions}

  top_suggestions = sorted(suggestions_probs,
                           key=suggestions_probs.get,
                           reverse=True)[:top_n]

  return top_suggestions

In [71]:
vocab = get_vocab(FILENAME)
probs = get_probs_dict(get_count_dict(preprocess_text_data(FILENAME)))

In [72]:
get_corrections('noice', probs, vocab, top_n=3)

['voice', 'noise', 'notice']

In [34]:
get_corrections('helloo', probs, vocab, top_n=3)

['hallow', 'fellow', 'hell']

In [36]:
get_corrections('noice', probs, vocab, top_n=5)

['noise', 'notice', 'nice', 'voice']

In [38]:
get_corrections('toit', probs, vocab, top_n=3)

['toil', 'tott']

In [66]:
def min_edit_distance(source, target, insert_cost=1, delete_cost=1, replace_cost=2):
  if source == target:
    return 0

  n_rows = len(source) + 1
  n_cols = len(target) + 1
  mat = np.zeros((n_rows, n_cols))

  for row in range(1, n_rows):
    mat[row][0] = mat[row - 1][0] + delete_cost

  for col in range(1, n_cols):
    mat[0][col] = mat[0][col - 1] + insert_cost

  for row in range(1, n_rows):
    for col in range(1, n_cols):
      r_cost = replace_cost
      if source[row - 1] == target[col - 1]:
        r_cost = 0

      mat[row][col] = min(mat[row - 1][col] + delete_cost,
                          mat[row][col - 1] + insert_cost,
                          mat[row - 1][col - 1] + r_cost)

  return mat[n_rows - 1][n_cols - 1]

In [73]:
def get_corrections_v1(word, vocab, top_n=3):
  min_edit_distances_dict = {w: min_edit_distance(word, w) for w in vocab}
  top_n_similar_words = list(sorted(min_edit_distances_dict,
                                    key=min_edit_distances_dict.get))[:top_n]

  return top_n_similar_words

In [74]:
get_corrections_v1('nice', vocab)

['nice', 'nicer', 'ice']

In [77]:
get_corrections_v1('helloo', vocab)

['hell', 'hoop', 'helm']

It looks like we don't have 'hello' in the text file.